In [ ]:
#Task 4: Data Integration via SQL

In [ ]:
#​ Joins the three tables into a unified dataset (structure based on common keys/columns)​
#Use the unified dataset to answer the following questions by means of SQL queries: ​

create VIEW unified_dataset.unified_dataset AS
SELECT 
    c.client_id,
    c.gender,
    c.age,
    c.province,
    COALESCE(c.income, 
        (SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY income) 
        FROM clients.clients WHERE province = c.province AND gender = c.gender)
    ) AS income,
    c.bank,
    c.payment,
    p.product_name AS lapse_product,
    l.has_lapsed,
    p.single_premium,
    p.couple_premium,
    p.family_premium
FROM 
    clients.clients c
LEFT JOIN 
    health_lapses.health_lapses l ON c.client_id = l.id
LEFT JOIN 
    health_products.health_products p ON l.client_code = p.product_name;
    
    

select * from unified_dataset.unified_dataset;



In [ ]:
#1 Are there any duplicate rows based on the id column, and if so, how many?​

SELECT 
    COUNT(*) - COUNT(DISTINCT client_id) AS duplicate_count
FROM 
    unified_dataset.unified_dataset;


In [ ]:
No, duplicate_count = 0

In [ ]:
#2​ What is the mean income per province and gender? If the income value is NULL,
#impute the NULL values. Provide motivation for your method of imputation. ​

SELECT 
    province,
    gender,
    ROUND(AVG(income)::numeric, 2) AS mean_income,
    COUNT(*) AS sample_size
FROM 
    unified_dataset.unified_dataset
GROUP BY 
    province, gender
ORDER BY 
    province, gender;


In [ ]:
How We were able to Handle Missing Income Data?:

We fillied in missing income numbers using the typical/median income for:

    The same province

    The same gender

This Works Best:

    Not skewed by extremes - Uses median (middle value) not average

    Local accuracy - Different areas/genders have different income patterns

    Better than guessing - More precise than using one number for all missing values

that is how we handled the problem.

In [ ]:
3# Which payment method is used by most customers per bank?

WITH payment_rankings AS (
    SELECT 
        bank,
        payment,
        COUNT(*) AS method_count,
        RANK() OVER (PARTITION BY bank ORDER BY COUNT(*) DESC) AS rank
    FROM 
        unified_dataset.unified_dataset
    GROUP BY 
        bank, payment
)
SELECT 
    bank,
    payment AS most_used_payment,
    method_count AS customers
FROM 
    payment_rankings
WHERE 
    rank = 1
ORDER BY 
    bank;


In [ ]:
Debit Order